In [3]:
import pandas as pd
import numpy as np

In [5]:
!wget https://raw.githubusercontent.com/conversationai/unhealthy-conversations/main/unhealthy_full.csv

--2023-01-21 16:46:48--  https://raw.githubusercontent.com/conversationai/unhealthy-conversations/main/unhealthy_full.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35005739 (33M) [text/plain]
Saving to: ‘unhealthy_full.csv.1’

unhealthy_full.csv. 100%[===================>]  33.38M  23.7MB/s    in 1.4s    

2023-01-21 16:46:51 (23.7 MB/s) - ‘unhealthy_full.csv.1’ saved [35005739/35005739]



In [7]:
data = pd.read_csv("unhealthy_full.csv")

In [10]:
ann_columns = data.columns.tolist()[4:]

In [11]:
', '.join(ann_columns)

'antagonize, condescending, dismissive, generalisation, generalisation_unfair, healthy, hostile, sarcastic'

In [12]:
for col in ann_columns:
    data[col] = data[col].astype(int)

In [25]:
np.random.seed(22)

selected_texts = data['comment'].drop_duplicates().sample(n=1000).tolist()
selected_data = data.loc[data['comment'].isin(selected_texts)]
not_selected_data = data.loc[~data['comment'].isin(selected_texts)]

In [58]:
major_votes_data = selected_data.groupby('comment')[ann_columns].mean()

major_votes_data[major_votes_data < 0.5] = 0
major_votes_data[major_votes_data >= 0.5] = 1
major_votes_data = major_votes_data.astype(int).reset_index()

def get_prompt(text):
    pattern = f'Which of the attributes: "antagonize", "condescending", "dismissive", "generalisation", "unfair generalisation", "healthy", "hostile", "sarcastic" describe a given text? Write your answer in the form of a Python list containing the appropriate attributes \n\n Text: {text}\n\n'

    return pattern

major_votes_data['comment'] = major_votes_data['comment'].apply(get_prompt)

In [59]:

def get_labels(row):
    return np.array(ann_columns)[row[ann_columns].values == 1]

major_votes_data['annotation'] = major_votes_data.apply(get_labels , axis=1)

In [61]:
major_votes_data.loc[:, ['comment', 'annotation']].to_csv("uc_prompts.csv")